# Aegis Ingestion Pipeline

The ingestion pipeline parses documents, chunks them, deduplicates, and
optionally stores results into vector databases and knowledge graphs.

This notebook demonstrates end-to-end ingestion with sample legal and
financial documents.

In [ ]:
from pathlib import Path

from aegis.ingestion.pipeline import IngestionConfig, IngestionPipeline

## 1. Configure the Pipeline

`IngestionConfig` controls chunk size, overlap, deduplication, and hashing.

In [ ]:
config = IngestionConfig(
    max_chunk_words=200,
    overlap_words=20,
    min_chunk_words=5,
    compute_hashes=True,
    deduplicate=True,
)
pipeline = IngestionPipeline(config=config)

print(f"Max chunk words: {config.max_chunk_words}")
print(f"Overlap words:   {config.overlap_words}")
print(f"Deduplication:   {config.deduplicate}")
print(f"Supported extensions: {pipeline.registry.supported_extensions()}")

## 2. Ingest a Legal Contract

In [ ]:
sample_dir = Path("../examples/sample_data")
contract_path = sample_dir / "contract_excerpt.txt"

contract_result = pipeline.ingest(contract_path)
print(f"Source:   {contract_result.source_path}")
print(f"Modality: {contract_result.modality.value}")
print(f"Chunks:   {contract_result.num_chunks}")
print(f"Doc ID:   {contract_result.document_id[:16]}...")

print("\nChunks:")
for i, chunk in enumerate(contract_result.chunks):
    preview = chunk.content[:80].replace('\n', ' ')
    word_count = chunk.metadata.get('word_count', len(chunk.content.split()))
    print(f"  [{i}] words={word_count:>3}, preview: {preview}...")

## 3. Ingest a Financial Filing

In [ ]:
filing_path = sample_dir / "financial_filing.txt"

filing_result = pipeline.ingest(filing_path)
print(f"Source:   {filing_result.source_path}")
print(f"Modality: {filing_result.modality.value}")
print(f"Chunks:   {filing_result.num_chunks}")

for i, chunk in enumerate(filing_result.chunks):
    preview = chunk.content[:80].replace('\n', ' ')
    print(f"  [{i}] {preview}...")

## 4. Batch Ingestion

Process multiple documents in a single call.

In [ ]:
batch_results = pipeline.ingest_batch([contract_path, filing_path])
print(f"Documents ingested: {len(batch_results)}")
for br in batch_results:
    print(f"  {Path(br.source_path).name}: {br.num_chunks} chunks ({br.modality.value})")

## 5. Ingestion History

In [ ]:
history = pipeline.history()
print(f"Total ingestions: {len(history)}")
for h in history:
    print(f"  [{h.document_id[:8]}...] {Path(h.source_path).name}: {h.num_chunks} chunks")

## 6. Storage Sinks

In production, ingested chunks are stored via sinks (PgVectorSink, Neo4jSink).
Without configured backends, `store_results` returns empty counts.

In [ ]:
store_counts = pipeline.store_results(contract_result, sinks=None)
print(f"Sink results: {store_counts}")
print("(No sinks configured -- in production, sinks write to pgvector, Neo4j, etc.)")

## 7. Entity Extraction (Preview)

The entity extractor identifies organisations, dates, financial terms,
and legal concepts from text. This feeds into the Neo4j knowledge graph.

In [ ]:
from aegis.ingestion.entity_extractor import EntityExtractor

extractor = EntityExtractor()

sample_text = (
    "Acme Corporation entered into a $2.5 million licensing agreement "
    "with GlobalTech Inc. on January 15, 2026. The contract includes "
    "an indemnification clause and a 3-year non-compete provision."
)

entities, relations = extractor.extract(sample_text)

print("Entities:")
for e in entities:
    print(f"  {e.label} ({e.entity_type})")

print("\nRelations:")
for r in relations:
    print(f"  {r.source} --[{r.relation}]--> {r.target}")

## 8. Deep Chunk Inspection

In [ ]:
if contract_result.chunks:
    chunk = contract_result.chunks[0]
    print(f"Chunk index:  {chunk.chunk_index}")
    print(f"Modality:     {chunk.modality.value}")
    print(f"Source:       {chunk.source_path}")
    print(f"Confidence:   {chunk.confidence}")
    print(f"Metadata:     {chunk.metadata}")
    print("Content:")
    print(f"  {chunk.content[:300]}...")

## Next Steps

- Configure pgvector + Neo4j with Docker: `docker compose up -d`
- Set `AEGIS_POSTGRES_DSN` and `AEGIS_NEO4J_URI` for real storage sinks
- Use the API: POST `/api/v1/ingestion/upload` for HTTP ingestion
- See `03_memory_operations.ipynb` for querying ingested data via memory
- Use `aegis data download --dataset cuad` to get real legal datasets